# 🧠 Proyecto ABP - Módulo 3: Obtención y preparación de datos

### 📍 Situación
El equipo de analítica de una empresa de e-commerce necesita estructurar un conjunto de datos desde archivos CSV, Excel y páginas web para dejarlos listos para análisis y modelos predictivos. Este proyecto sigue 6 lecciones para llegar a un dataset limpio y estructurado.


### 🔧 Lección 1 - Generación de datos con NumPy

In [1]:
import numpy as np

# Crear arrays ficticios
clientes_id = np.arange(1000, 1010)
compras = np.random.randint(1, 5, size=10)
montos = np.random.uniform(10000, 200000, size=10).round(2)

# Guardar en archivo .npy
np.save('datos_clientes.npy', [clientes_id, compras, montos])

# Ver ejemplo de operaciones
print("Total de compras:", np.sum(compras))
print("Monto promedio:", np.mean(montos))


Total de compras: 24
Monto promedio: 101880.00700000001


### 📄 Lección 2 - Exploración inicial con Pandas

In [2]:
import pandas as pd

# Cargar datos desde archivo NumPy
clientes_id, compras, montos = np.load('datos_clientes.npy', allow_pickle=True)
df = pd.DataFrame({
    "ClienteID": clientes_id,
    "Compras": compras,
    "MontoTotal": montos
})

# Exploración básica
print(df.head())
print(df.describe())
print(df[df["MontoTotal"] > 150000])

# Guardar como CSV
df.to_csv("clientes_datos.csv", index=False)


   ClienteID  Compras  MontoTotal
0     1000.0      2.0   126942.84
1     1001.0      4.0    10183.73
2     1002.0      1.0   193356.46
3     1003.0      2.0   132348.04
4     1004.0      1.0   158004.77
        ClienteID    Compras     MontoTotal
count    10.00000  10.000000      10.000000
mean   1004.50000   2.400000  101880.007000
std       3.02765   1.074968   57888.398005
min    1000.00000   1.000000   10183.730000
25%    1002.25000   2.000000   66034.865000
50%    1004.50000   2.000000  101443.750000
75%    1006.75000   3.000000  142973.470000
max    1009.00000   4.000000  193356.460000
   ClienteID  Compras  MontoTotal
2     1002.0      1.0   193356.46
4     1004.0      1.0   158004.77


### 📑 Lección 3 - Integración de fuentes (CSV, Excel y Web)

In [3]:
# CSV
df_csv = pd.read_csv("clientes_datos.csv")

# Excel (simulación)
df_excel = pd.DataFrame({
    "ClienteID": [1000, 1001, 1002],
    "Ciudad": ["Santiago", "Valparaíso", "Concepción"]
})
df_excel.to_excel("info_adicional.xlsx", index=False)
df_excel = pd.read_excel("info_adicional.xlsx")

# Web (simulación con tabla simple)
from io import StringIO

html = """
<table>
<tr><th>ClienteID</th><th>Segmento</th></tr>
<tr><td>1000</td><td>Premium</td></tr>
<tr><td>1001</td><td>Estándar</td></tr>
</table>
"""
df_web = pd.read_html(StringIO(html))[0]

# Merge final
df_merge = df_csv.merge(df_excel, on="ClienteID", how="left").merge(df_web, on="ClienteID", how="left")
df_merge.to_csv("clientes_consolidados.csv", index=False)


### 🧼 Lección 4 - Valores nulos y outliers

In [4]:
# Revisión de valores nulos
print(df_merge.isnull().sum())

# Imputar segmento faltante como "Desconocido"
df_merge["Segmento"].fillna("Desconocido", inplace=True)

# Detección de outliers (Z-score)
from scipy.stats import zscore
df_merge["Z_Monto"] = zscore(df_merge["MontoTotal"])
df_sin_outliers = df_merge[df_merge["Z_Monto"].abs() < 3]
df_sin_outliers.drop(columns="Z_Monto", inplace=True)

# Guardar resultado
df_sin_outliers.to_csv("clientes_limpios.csv", index=False)


ClienteID     0
Compras       0
MontoTotal    0
Ciudad        7
Segmento      8
dtype: int64


### 🧹 Lección 5 - Data Wrangling

In [5]:
# Convertir tipos si es necesario
df_wrangle = df_sin_outliers.copy()
df_wrangle["Compras"] = df_wrangle["Compras"].astype("int")

# Nueva columna
df_wrangle["PromedioCompra"] = (df_wrangle["MontoTotal"] / df_wrangle["Compras"]).round(2)

# Usar lambda/map
df_wrangle["TipoCliente"] = df_wrangle["Compras"].map(lambda x: "Frecuente" if x > 2 else "Ocasional")

# Guardar
df_wrangle.to_csv("clientes_wrangle.csv", index=False)


### 📊 Lección 6 - Agrupamiento y pivoteo

In [6]:
# Agrupamiento por ciudad
resumen = df_wrangle.groupby("Ciudad").agg({
    "MontoTotal": ["mean", "sum"],
    "Compras": "sum"
}).round(2)

# Pivot (si fuera necesario)
pivot = df_wrangle.pivot_table(values="MontoTotal", index="Ciudad", columns="Segmento", aggfunc="mean")

# Melt de ejemplo
melted = pd.melt(df_wrangle, id_vars=["ClienteID"], value_vars=["MontoTotal", "PromedioCompra"])

# Guardar resultados
df_wrangle.to_excel("clientes_final.xlsx", index=False)


### 📝 Documento resumen

#### Herramientas utilizadas:
- NumPy para la generación y procesamiento eficiente de arrays numéricos.
- Pandas para exploración, manipulación, limpieza y exportación de datos.

#### Fuentes de datos:
- CSV: Datos de clientes.
- Excel: Información adicional como ciudad.
- Web: Segmento de clientes (simulada).

#### Procesamiento:
- Se detectaron y trataron valores nulos (`fillna`).
- Se eliminaron outliers con Z-score.
- Se transformaron tipos de datos y se agregaron nuevas columnas con `apply`, `map`, `lambda`.
- Se agruparon datos y se usó `pivot` y `melt` para reestructuración.

#### Resultado:
- Dataset final limpio, exportado como CSV y Excel, listo para modelos predictivos o dashboards.
